<a href="https://colab.research.google.com/github/carlos-alves-one/-ML-Zoomcamp-Week-5/blob/main/ML_Zoomcamp_Week_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Goldsmiths University of London
**Author....: Carlos Manuel de Oliveira Alves**<br>
**Student..: cdeol003**<br>
**Created..: 03/10/2022**

In [1]:
# Import all necessay libraries that we will use in this project
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Import the library warnings to ignore the warnings from the system
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Data Preparation:

# Read the dataset, store it in dataframe
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Make the categorical data of the dataframe consistent 
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for col in categorical_columns:
  df[col] = df[col].str.lower().str.replace(' ', '_')

# Convert a serie of the dataframe to a number
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

# Fill the missing values of the total charges serie with zeros
df.totalcharges = df.totalcharges.fillna(0)

# Update the churn data with numbers
df.churn = (df.churn == 'yes').astype(int)

In [3]:
# Define the sizes of the datasets with 20% and use random state so the results are reproducible
# the full train dataset has 80% and the test 20%
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)

In [ ]:
# Create a list with numerical variables from the dataframe
numerical = ['tenure','monthlycharges','totalcharges']